# Stock trades data in BigQuery

In this notebook we'll
- inspect stock trades data that we have in BigQuery;
- craft the query to get open, close, minimum and maximum values for a given stock symbol and defined time interval and window size;
- plot the candlestick chart

### Import modules

In [ ]:
from google.cloud import bigquery
import plotly.graph_objects as go
import datetime
import time
import ipywidgets as widgets
from IPython.core.display import display

### Setting constants:
- BQ_PROJECT_ID - Google Cloud project ID where dataset resides
- DEFAULT_SYMBOL - default stock symbol on which we'll be filtering trades.
- TIME_WINDOW - window for which we'll be calculating candles.

In [ ]:
# Put you project ID here
BQ_PROJECT_ID = '<PROJECT ID>'
DEFAULT_SYMBOL = 'GE'
TIME_WINDOW = 300 # in seconds

### Inspect data in BigQuery

In [ ]:
client = bigquery.Client(project = BQ_PROJECT_ID)
df = client.query('SELECT * FROM `polygon.trades` LIMIT 5').to_dataframe()
df

### Inspect what symbols are in dataset

In [ ]:
symbols = client.query('SELECT DISTINCT symbol FROM `polygon.trades` ORDER BY symbol').to_dataframe()

# output all symbols
print(symbols)

# output filtered symbols. e.g. characters starting with ZX
print(symbols[symbols.symbol.str.match('GE.*')])

### Select stock symbol you want to analyze

In [ ]:
symbol = DEFAULT_SYMBOL

### Inspect what records do we have for a given symbol

In [ ]:
info = client.query('''
  SELECT 
    MIN(UNIX_SECONDS(ts)) as min_ts, MAX(UNIX_SECONDS(ts)) as max_ts, COUNT(*) as total
  FROM `polygon.trades` WHERE symbol="{}"'''
  .format(symbol)).to_dataframe()

min_date = datetime.datetime.fromtimestamp(info['min_ts'][0])
max_date = datetime.datetime.fromtimestamp(info['max_ts'][0])

print("Info for %s:\n\tMin date: %s \n\tMax date: %s \n\tRows: %d\n" % (symbol, min_date, max_date, info['total']))

### Define date range

In [ ]:
start_date_widget = widgets.DatePicker(
    description='Start Date',
    value=min_date,
    disabled=False
)
finish_date_widget = widgets.DatePicker(
    description='Finish Date',
    value=max_date,
    disabled=False
)
display(start_date_widget, finish_date_widget)

### Query min, max,open and close prices over time windows

In [ ]:
start_date = datetime.datetime.combine(start_date_widget.value, datetime.time(0,0,0)) if start_date_widget.value else datetime.datetime.combine(min_date, datetime.time(0,0,0))
finish_date = datetime.datetime.combine(finish_date_widget.value, datetime.time(23,59,59)) if start_date_widget.value else datetime.datetime.combine(max_date, datetime.time(23,59,59))

ts_start_date = int(time.mktime(start_date.timetuple()))
ts_finish_date = int(time.mktime(finish_date.timetuple()))

df = client.query('''
  SELECT
    UNIX_SECONDS(min_ts) AS min_ts, UNIX_SECONDS(max_ts) AS max_ts,
    min_price, max_price, ti1.price AS open_price, ti2.price AS close_price
  FROM (
    SELECT
      MIN(ts) AS min_ts, MAX(ts) AS max_ts, 
      MIN(price) AS min_price, MAX(price) AS max_price, 
      FLOOR((UNIX_SECONDS(ts)/{1}))
    FROM polygon.trades
    WHERE symbol="{0}" AND UNIX_SECONDS(ts)>={2} AND UNIX_SECONDS(ts)<={3}
    GROUP BY FLOOR((UNIX_SECONDS(ts)/{1}))
  ) q
  JOIN polygon.trades ti1 ON ti1.ts=min_ts AND ti1.symbol="{0}"
  JOIN polygon.trades ti2 ON ti2.ts=max_ts AND ti2.symbol="{0}"
  ORDER BY min_ts
'''.format(symbol, TIME_WINDOW, ts_start_date, ts_finish_date)).to_dataframe()

df['date'] = df.apply(lambda row: datetime.datetime.fromtimestamp(row.min_ts), axis = 1)
df.head()

### Plot candlestick chart

In [ ]:
# STEP 6

fig = go.Figure(data=[go.Candlestick(x=df['date'],
                open=df['open_price'], high=df['max_price'],
                low=df['min_price'], close=df['close_price'])
                     ])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()